## Load packages

In [11]:
#!python -m spacy download en_core_web_lg
# !conda install spacy download en_core_web_lg
# !python -m spacy validate
import pandas as pd
import spacy.displacy as displacy
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
import random    
from collections import Counter
import os
random.seed(42)


In [19]:
os.getcwd()

'/Users/rahul/Desktop/Food-Pulse/src/entity'

## Helper Functions

In [12]:
# Key words in text
start = '<START:food>'
end = '<END>'

replace_vals = {start:'', end:''}
# Function to replace keywords with ''
def replace_all(text, dic):
    '''
    Replace all the words in the text with the items from the dictionary
    '''
    for i,j in dic.items():
        text = text.replace(i,j)
    return text

In [13]:
def find_nth(haystack, needle, n):
    '''
    Find the location of a duplicate word in a sentence
    '''
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

## Train data

In [4]:
train_list = []

In [5]:
# Reading in first X lines of the text file
count = 0
num_lines = 1000

# If it's below the numbers threshold, randomly sample 5% of the data
# otherwise add the numbers between the alloted threshold

with open('review_train', 'r') as f:
    for line in f:
        if count == num_lines:
            break
        print(line, sep = '\n')
        train_list.append(line)
        count +=1
print(count)


the  <START:food>  burgers <END>  at the sport bowl are 

At lunch, you can get a  <START:food>  burger <END>  fries 

 <START:food> soda <END>  for eight bucks 

i can see some people thinking the  <START:food>  burgers <END>  are too greasy, the grease gives you a delightful contented, weighed down, and fuzzy brained feeling 

a quick rundown on other items I've tried the  <START:food>  meatball <END>  sub was so so, huge 

Go to gino's a mile away if you're craving  <START:food>  meatballs <END>  

the  <START:food>  patty <END>  melt is just a  <START:food>  burger <END>  on bread, so obviously it rules 

the  <START:food>  pizza <END>  is decent, with a crust that tastes like pizza hut's  <START:food>  deep dish <END>  

the  <START:food>  fish <END>  fry is a lot of good food for a good price 

the best part about the sport bowl is that if you feel some postprandial guilt for your frenzied ingestion of large amounts of fat and grease, you can pop over to willy west next door to 

In [6]:
TRAIN_DATA = []
count_edge_case = 0
# Iterate through each review
for sentence in train_list:
    # Get rid of leading/trailing spaces plus '\n'
    sentence = sentence.strip()
    
    # Code to get the all values inside '<START:food>' and '<END>'
    split = sentence.split(start)
    food_items = []
    for i in range(0,len(split)-1):
        food_items.append(split[i+1].split(end)[0].strip())
    
    # Removing all marker Keywords, removing extra spaces, and sentence lowercase
    sentence = replace_all(sentence, replace_vals)
    sentence = ' '.join(sentence.split())
    sentence = sentence.lower()
    
    # Creating the format necessary for spacy custom NER
    entity_list = []
    food_entities = Counter(food_items)
    for k,v in food_entities.items():
        # Multiple instances of a food item
        if v > 1:
            for val in range(0,v):
                ind_start = find_nth(sentence,k,val+1)
                ind_end = ind_start + len(k)
                
                entity_list.append((ind_start,ind_end, 'FOOD'))
        # Single instace of a food item
        else:
            ind_start = sentence.find(k)
            ind_end = ind_start + len(k)
            entity_list.append((ind_start, ind_end, 'FOOD') )
    
    # Case where food entities are subsets of other entities in sentence
    check_locations = []
    for val in entity_list:
        check_locations.append((val[0],val[1]))
        
    edge_case = False
    for ind, i in enumerate(check_locations):
        if len(check_locations) == 1:
            break
        else:
            for ind1,j in enumerate(check_locations):
                if ind == ind1:
                    continue
                else:
                    x = range(check_locations[ind][0], check_locations[ind][1])
                    y = range(check_locations[ind1][0], check_locations[ind1][1])

                    xs = set(x)

                    if(len(xs.intersection(y)) > 1):
                        edge_case = True
                        #print(x,y)
                        #print('found mismatch')
    if edge_case == True:
        count_edge_case += 1
        continue
    
    new_format = (sentence, {'entities': entity_list})
    TRAIN_DATA.append(new_format)
print(count_edge_case)

5


## Define variables

In [14]:
model = os.getcwd() + '/output_dir'
output_dir= os.getcwd() + '/output_dir'
n_iter=1

In [8]:
output_dir

'/Users/rahul/Desktop/git projects/Food-Pulse/src/entity/output_dir'

## Load the model

In [15]:
if model is not None:
    nlp = spacy.loead(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Loaded model '/Users/rahul/Desktop/Food-Pulse/src/entity/output_dir'


## Setup pipeline

In [10]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

## Train the recognizer

In [39]:
for _, annotations in TRAIN_DATA:
    #print(_, annotations)

    for ent in annotations.get('entities'):
        #print(ent)
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  proc.begin_training(
  0%|          | 112/301654 [00:03<3:14:45, 25.80it/s]/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "i was disappointed they wouldn't let me use my own..." with entities "[(51, 54, 'FOOD')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. 

{'ner': 250526.9530690142}


## Test the trained recognizer

In [11]:
df = pd.read_csv('reviews.csv')

In [12]:
df.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [13]:
# TEST_DATA = list(df.loc[0:3,'text'])

In [14]:
# TEST_DATA

In [16]:
TEST_DATA = [
    'Taco Dale is my go to place for having Mexican food, we love their burrito, Tacos, basically everything They have a great staff who hears you out when placing the order and cooks it perfect as per your needs',
    "My new favorite place I get tacos from around the area and today was my 3rd day in a row, here. All I've had are the tacos. If they're this great, I can't even imagine the rest of the menu! Great restaurant!",
    'My favorite "fast food " Mexican restaurant! Always hot, fresh and tasty!!! The eclairs and shrimp tacos are the best here never dry or overcooked like some other restaurants do.',
    'They have always had the very top tier, best fortune cookies I’ve ever found. They also have the chicken fried rice, which is also really good. ',
    'Good flavor but way overpriced and portions are very small. The crab rangoon is almost all bread, with a miniscule amount of filling. The white rice portion is not even enough to feed one person',
    "Disgusting food!I'm not sure how they have 4.3 rating.The presentation is bad, the taste is bad, General Tso's chicken doesn't even taste like chicken. I'm never going there again",
    'I had ordered 3 lunch specials  that included pastries. Nice portion size btw. Ordered the Orange Chicken, Mongolian Beef, Beef and Broccoli. The Orange Chicken was delicious. Will definitely order again. The Mongolian Beef was kinda weird tasting and had way more onions then beef. not worth getting, unless you want just the onions. :( The Beef and Broccoli was also delicious. A little salty, if your watching your sodium I suggest you skip this dish. However it was good. Oh for got to mention the meat on both beefs where very chewy for some reason. My kids liked the rice, however I didnt enjoy it. To many chunks of onions. I like the fried rice with bean sprouts. Also no egg roll or pop with lunch specials in case you wanted to know. '
]
# Convert everything to lowercase
TEST_DATA = list(map(str.lower,TEST_DATA))

In [17]:
TEST_DATA

['taco dale is my go to place for having mexican food, we love their burrito, tacos, basically everything they have a great staff who hears you out when placing the order and cooks it perfect as per your needs',
 "my new favorite place i get tacos from around the area and today was my 3rd day in a row, here. all i've had are the tacos. if they're this great, i can't even imagine the rest of the menu! great restaurant!",
 'my favorite "fast food " mexican restaurant! always hot, fresh and tasty!!! the eclairs and shrimp tacos are the best here never dry or overcooked like some other restaurants do.',
 'they have always had the very top tier, best fortune cookies i’ve ever found. they also have the chicken fried rice, which is also really good. ',
 'good flavor but way overpriced and portions are very small. the crab rangoon is almost all bread, with a miniscule amount of filling. the white rice portion is not even enough to feed one person',
 "disgusting food!i'm not sure how they have 

In [16]:
# print(os.getcwd())
# df = pd.read_json('/Users/rahul/Desktop/Food-Pulse/src/entity/scrapped_restaurant_reviews/BabaKabab.json')

In [17]:
# df.head()

In [25]:
# TEST_DATA = []
# for val in df.reviews:
#     TEST_DATA.append(val['review'].strip())

In [3]:
# TEST_DATA

In [18]:
# TEST_DATA = [
#     ('Anytime I am in Chicago I always get the cheese fries, but this time I chose to get the laptops and I loved it', {
#         'entities': [(0, 25, 'FOOD')]
#     }),
#      ('i like to eat butter chicken', {
#         'entities': [(0, 12, 'FOOD')]
#     }),
#     ('The double ka meetha is awesome here', {
#         'entities': [(0, 9, 'FOOD')]
#     })
# ]

# TEST_DATA = [
#     'The chicken eggs benedict was dope',
#     'Have to try the coffee here',
#     'OMG The eclairs here are to DIE FOR',
#     'Best snacks in town. Must try'
# ]
for text in TEST_DATA:
    doc = nlp(text)
    #print(doc)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('taco dale', 'FOOD')]
Entities [('tacos', 'FOOD'), ('tacos', 'FOOD')]
Entities [('shrimp', 'FOOD')]
Entities [('fortune', 'FOOD'), ('chicken', 'FOOD')]
Entities [('crab', 'FOOD'), ('white rice', 'FOOD')]
Entities [('4.3', 'FOOD'), ('chicken', 'FOOD'), ('chicken', 'FOOD')]
Entities [('pastries', 'FOOD'), ('orange', 'FOOD'), ('beef', 'FOOD'), ('orange', 'FOOD'), ('beef', 'FOOD'), ('beef', 'FOOD'), ('beef', 'FOOD'), ('meat', 'FOOD'), ('fried rice', 'FOOD'), ('bean sprouts', 'FOOD'), ('egg roll', 'FOOD')]


In [ ]:
'Taco Dale is my go to place for having Mexican food, we love their burrito, Tacos, basically everything They have a great staff who hears you out when placing the order and cooks it perfect as per your needs',
    "My new favorite place I get tacos from around the area and today was my 3rd day in a row, here. All I've had are the tacos. If they're this great, I can't even imagine the rest of the menu! Great restaurant!",
    'My favorite "fast food " Mexican restaurant! Always hot, fresh and tasty!!! The eclairs and shrimp tacos are the best here never dry or overcooked like some other restaurants do.',
    'They have always had the very top tier, best fortune cookies I’ve ever found. They also have the chicken fried rice, which is also really good. ',
    'Good flavor but way overpriced and portions are very small. The crab rangoon is almost all bread, with a miniscule amount of filling. The white rice portion is not even enough to feed one person',
    "Disgusting food!I'm not sure how they have 4.3 rating.The presentation is bad, the taste is bad, General Tso's chicken doesn't even taste like chicken. I'm never going there again",
    'I had ordered 3 lunch specials  that included pastries. Nice portion size btw. Ordered the Orange Chicken, Mongolian Beef, Beef and Broccoli. The Orange Chicken was delicious. Will definitely order again. The Mongolian Beef was kinda weird tasting and had way more onions then beef. not worth getting, unless you want just the onions. :( The Beef and Broccoli was also delicious. A little salty, if your watching your sodium I suggest you skip this dish. However it was good. Oh for got to mention the meat on both beefs where very chewy for some reason. My kids liked the rice, however I didnt enjoy it. To many chunks of onions. I like the fried rice with bean sprouts. Also no egg roll or pop with lunch specials in case you wanted to know. '

In [46]:
# for text, _ in TRAIN_DATA:
#     doc = nlp(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

## Save model

In [47]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /Users/rahul/Desktop/Food-Pulse/src/entity/output_dir


## Test the saved model

In [48]:
# print("Loading from", output_dir)
# nlp2 = spacy.load(output_dir)
# for text, _ in TRAIN_DATA:
#     doc = nlp2(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from /Users/rahul/Desktop/Food-Pulse/src/entity/output_dir
Entities [('pancakes', 'FOOD')]
Entities [('bourbon', 'FOOD')]
Entities [('flamingo', 'FOOD')]
Entities [('spinach', 'FOOD')]
Entities [('rice', 'FOOD')]
Entities []
Entities [('curry', 'FOOD')]
Entities [('burger', 'FOOD')]
Entities [('chicken', 'FOOD'), ('taco', 'FOOD')]
Entities [('burger', 'FOOD'), ('burgers', 'FOOD')]
Entities [('pancakes', 'FOOD')]
Entities [('peanut', 'FOOD')]
Entities [('tea', 'FOOD')]
Entities [('cookie', 'FOOD')]
Entities [('mozzarella', 'FOOD')]
Entities [('hummus', 'FOOD'), ('chicken', 'FOOD')]
Entities [('beans', 'FOOD'), ('penne', 'FOOD')]
Entities [('fajitas', 'FOOD'), ('enchiladas', 'FOOD')]
Entities [('taco', 'FOOD')]
Entities [('pizza', 'FOOD')]
Entities [('broth', 'FOOD')]
Entities [('rice', 'FOOD')]
Entities [('salad', 'FOOD'), ('shrimp', 'FOOD'), ('shrimp', 'FOOD')]
Entities [('fish', 'FOOD')]
Entities [('fish', 'FOOD')]
Entities [('spring rolls', 'FOOD'), ('sticky rice', 'FOOD'), (

KeyboardInterrupt: 

In [7]:
# Possible ideas. 

# I don't thinking I am training the group of words. Only single word entities.
# Look at IEPY open source nlp active learning package to use GUI to annotate data 
# If we had budget, use prodigy.
# Train new model with pattern matching on all possible food combinations?
# Idea being you don't have to do levansthein distance if we wanted to brute force all nouns